In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
URL = "https://www.mwis.org.uk/forecasts/scottish/southern-uplands/text"
html = requests.get(URL).text

In [3]:
soup = BeautifulSoup(html, "html.parser")

In [4]:
soup.find("div", class_="forecast-area")

<div class="forecast-area">
<h1>Southern Uplands</h1>
<p>The Galloway hills eastward to the Lammermuir hills. The Cheviots (including higher hills within the adjacent Northumberland NP). </p>
<h4>
Today's            Forecast
        </h4>
<div class="row">
<div class="col-xs-12" style="width:100%;">
<div class="icon-modal-content" id="modal-xs-0">
<span class="close" id="cross-0" onclick="CloseIconModal()">×</span>
<h3>Moderate breeze, locally gusty</h3>
<img alt="Moderate breeze, locally gusty" class="icon-img" height="200" src="https://www.mwis.org.uk/assets/weather-icons/Wind-Fairly_Small.jpg" title="Moderate breeze, locally gusty" width="200"/>
<p>A noticeable breeze, feeling cool where exposed on tops. Locally may be gusty, but unlikely to affect walking progress. 15-25mph.</p>
</div>
<div class="icon-modal-content" id="modal-xs-1">
<span class="close" id="cross-1" onclick="CloseIconModal()">×</span>
<h3>Clear</h3>
<img alt="Clear" class="icon-img" height="200" src="https://www.mw

In [5]:
soup.find("h1").text

'Southern Uplands'

In [6]:
soup.find("p").text

'The Galloway hills eastward to the Lammermuir hills. The Cheviots (including higher hills within the adjacent Northumberland NP). '

In [7]:
divs = soup.find_all("div", class_= "row category-end")

In [8]:
divs[0]

<div class="row category-end">
<div class="col-xs-12 col-lg-5 col-xl-4"><h4>Headline for Southern Uplands</h4></div>
<div class="col"><p>Bright sunshine. Wind dropping. Slight frost at first.</p></div>
</div>

In [9]:
soup.find_all("div", class_= "col")

[<div class="col">
 <p>
 <strong>Southern Uplands</strong><br/>
 <strong>Tuesday 25th November 2025</strong><br/>
 <small>
                                     Last updated
                                     Mon 24th Nov 25 at
                                     12:15PM
                                 </small>
 </p>
 </div>,
 <div class="col"><p>Higher areas, mainly Scottish Highlands frozen, and frost some valleys at first. 
 Otherwise, benign as from the west the northerly wind drops and snow and hail showers become isolated. </p></div>,
 <div class="col"><p>Bright sunshine. Wind dropping. Slight frost at first.</p></div>,
 <div class="col"><p>Northerly 20 to 35mph, strongest Borders, gradually easing from west to 10 to 15mph. </p></div>,
 <div class="col"><p>Considerable buffeting and wind chill after dawn, but will become small through afternoon.</p></div>,
 <div class="col">
 <p><strong>Precipitation very unlikely</strong></p>
 <p>Risk of a  hail or snow flurries or showers ne

In [10]:
date_updated = soup.find_all("small")[0].text.strip()

In [11]:
# forecast date
# headline
# how windy
# wind effect
# how wet
# cloud on the hills
# chance of cloud free
# sunshine and air clarity
# how cold
# freezing level


In [12]:
day0 = soup.find_all("div", id="Forecast0")
day1 = soup.find_all("div", id="Forecast1")
day2 = soup.find_all("div", id="Forecast2")

In [13]:
soup.find_all("strong")

[<strong>Forecast information for…</strong>,
 <strong>Southern Uplands</strong>,
 <strong>Tuesday 25th November 2025</strong>,
 <strong>Precipitation very unlikely</strong>,
 <strong>Mostly or all clearing</strong>,
 <strong>Southern Uplands</strong>,
 <strong>Wednesday 26th November 2025</strong>,
 <strong>Rain and drizzle; extensive west</strong>,
 <strong>Most hills filling in with fog</strong>,
 <strong>Southern Uplands</strong>,
 <strong>Thursday 27th November 2025</strong>,
 <strong>Frequent or constant rain</strong>,
 <strong>All or nearly all summits persistently foggy.</strong>]

In [14]:
def clean_string(s: str) -> str:
    return " ".join(s.split()).strip()

In [15]:
#forecast0 = tomorrow
headers = soup.find("div", id="Forecast0").find_all("h4")
res = {header.text: clean_string(header.find_next("p").text) for header in headers}
res.pop("Viewing Forecast For")
res.pop("Summary for all mountain areas")
res

{'Headline for Southern Uplands': 'Bright sunshine. Wind dropping. Slight frost at first.',
 'How windy? (On the summits)': 'Northerly 20 to 35mph, strongest Borders, gradually easing from west to 10 to 15mph.',
 'Effect of the wind on you?': 'Considerable buffeting and wind chill after dawn, but will become small through afternoon.',
 'How Wet?': 'Precipitation very unlikely',
 'Cloud on the hills?': 'Mostly or all clearing',
 'Chance of cloud free summits?': '90% by afternoon',
 'Sunshine and air clarity?': 'Bright sunshine all day, although patchy cloud The Cheviot. The air very clear so excellent visibility, but fog some valleys and higher slopes.',
 'How Cold? (at 750m)': '1C',
 'Freezing Level': 'Just above the summits Frost many valleys morning.'}

In [16]:
res["Last Updated"] = clean_string(soup.find("div", id="Forecast0").find("small").text).replace("Last updated ", "")
res["Forecast Area"] = soup.find("div", class_="forecast-area").find("h1").text

In [17]:
import json
json.dumps(res)

'{"Headline for Southern Uplands": "Bright sunshine. Wind dropping. Slight frost at first.", "How windy? (On the summits)": "Northerly 20 to 35mph, strongest Borders, gradually easing from west to 10 to 15mph.", "Effect of the wind on you?": "Considerable buffeting and wind chill after dawn, but will become small through afternoon.", "How Wet?": "Precipitation very unlikely", "Cloud on the hills?": "Mostly or all clearing", "Chance of cloud free summits?": "90% by afternoon", "Sunshine and air clarity?": "Bright sunshine all day, although patchy cloud The Cheviot. The air very clear so excellent visibility, but fog some valleys and higher slopes.", "How Cold? (at 750m)": "1C", "Freezing Level": "Just above the summits Frost many valleys morning.", "Last Updated": "Mon 24th Nov 25 at 12:15PM", "Forecast Area": "Southern Uplands"}'

In [18]:
from datetime import date
import re

def get_forecast_date(soup: BeautifulSoup, forecast_day: str) -> date:
    """NOT TESTED"""
    headers = soup.find("div", id=forecast_day).find_all("h4")
    date_str = {header.text: header.find_next("p").text for header in headers}["Viewing Forecast For"].splitlines()[2]
    clean_date = re.sub(r"(\d+)(st|nd|rd|th)", r"\1", date_str)
    return date.strptime(clean_date, "%A %d %B %Y")

In [19]:
get_forecast_date(soup, "Forecast2")

datetime.date(2025, 11, 27)

In [20]:
def scrape(region: str) -> dict[str, str]:
    """Scrape MWIS for forecast"""
    URL = f"https://www.mwis.org.uk/forecasts/scottish/{region}/text"
    html = requests.get(URL).text
    soup = BeautifulSoup(html, "html.parser")

    forecast = {}

    for forecast_day in ["Forecast0", "Forecast1", "Forecast2"]:

        forecast_date = get_forecast_date(soup, forecast_day=forecast_day)

        headers = soup.find("div", id=forecast_day).find_all("h4")
        forecast[forecast_date] = {header.text: clean_string(header.find_next("p").text) for header in headers}

        forecast[forecast_date]["Last Updated"] = clean_string(soup.find("div", id="Forecast0").find("small").text).replace("Last updated ", "")
        forecast[forecast_date]["Forecast Area"] = soup.find("div", class_="forecast-area").find("h1").text


        # superfluous data
        forecast[forecast_date].pop("Viewing Forecast For")
       # forecast[forecast_date].pop("Summary for all mountain areas")

    return forecast
    

In [21]:
scrape("southern-uplands")

{datetime.date(2025, 11, 25): {'Summary for all mountain areas': 'Higher areas, mainly Scottish Highlands frozen, and frost some valleys at first. Otherwise, benign as from the west the northerly wind drops and snow and hail showers become isolated.',
  'Headline for Southern Uplands': 'Bright sunshine. Wind dropping. Slight frost at first.',
  'How windy? (On the summits)': 'Northerly 20 to 35mph, strongest Borders, gradually easing from west to 10 to 15mph.',
  'Effect of the wind on you?': 'Considerable buffeting and wind chill after dawn, but will become small through afternoon.',
  'How Wet?': 'Precipitation very unlikely',
  'Cloud on the hills?': 'Mostly or all clearing',
  'Chance of cloud free summits?': '90% by afternoon',
  'Sunshine and air clarity?': 'Bright sunshine all day, although patchy cloud The Cheviot. The air very clear so excellent visibility, but fog some valleys and higher slopes.',
  'How Cold? (at 750m)': '1C',
  'Freezing Level': 'Just above the summits Fros

In [ ]:
# GET /headlines/{region} 
# GET /forecasts/{region}?days_from_now=
# GET /forecasts/ (get all)

In [ ]:
# pull from MWIS once an hour
# save in DB - DuckDB?